## Задание 1

Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

In [1]:
import pandas as pd
import re

In [2]:
ratings = pd.read_csv('ml-latest/ratings.csv')
movies_ratings = ratings.merge(pd.read_csv('ml-latest/movies.csv'), on='movieId', how='left')
r = movies_ratings[movies_ratings['rating'] == 5.0].filter(items=['title', 'rating']).groupby('title').count()
display(r.sort_values(by='rating', ascending=False).iloc[0])

rating    153
Name: Shawshank Redemption, The (1994), dtype: int64

## Задание 2

По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [3]:
power = pd.read_csv('power.csv')

In [5]:
вaltic_countries_c4_c12_c21 = power[((power['country'] == 'Lithuania') | 
                                     (power['country'] == 'Latvia') | 
                                     (power['country'] == 'Estonia')) 
                                    &
                                    ((power['category'] == 4) |
                                     (power['category'] == 12) |
                                     (power['category'] == 21))
                                    &
                                    (power['quantity'] >= 0)
                                    &
                                    (power['year'] >= 2005) 
                                    &
                                    (power['year'] <= 2010)
                                   ]
вaltic_countries_quantity_sum = вaltic_countries_c4_c12_c21[['country', 'quantity']].groupby('country').sum()
display(вaltic_countries_quantity_sum)

,quantity
country,
Estonia,63440.0
Latvia,54844.0
Lithuania,122296.0


## Задание 3

Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni

In [23]:
def currency_code_filter(curr_code:str) -> str:
    """
    Функция для очистки кода валюты от лишних символов
    
    Например, "JPY/EUR(100:1)" будет преобразовано в "JPY/EUR"
    
    Входные параметры:
    val_code -- код валюты для очистки, тип str
    
    Выходные значения: отфильтрованный код валюты, тип str
    """
    pattern = r'(\w{3}/\w{3})'
    result = re.compile(pattern).findall(curr_code)
    assert len(result) == 1, f'Функция не смогла корректно распознать код валюты {curr_code}'
    
    return result[0]

In [24]:
html = pd.read_html('https://www.finanz.ru/valyuty/v-realnom-vremeni', 
                    encoding='utf-8',
                    attrs={'class': 'quote_list'})[0]
# Удаляем столбец, в котором были графические ссылки
del(html['Unnamed: 0'])
# Переименовываем столбцы
html.columns = ['Валюта','Предложение','Спрос','Предыдущее закрытие','%','Абсолютное значение','Время']
# Удаляем некорректные строки, которые содержат названия вложенных таблиц
html = html[html['Время'] != 'Время']
# Изменяем коды валют, удаляем все лишние символы
html['Валюта'] = html['Валюта'].transform(currency_code_filter)
# Индексируем все Series по валюте
html.index = html['Валюта']
# Удаляем стобец с валютами, теперь он у нас в индексе
del(html['Валюта'])

In [25]:
display(html)

,Предложение,Спрос,Предыдущее закрытие,%,Абсолютное значение,Время
Валюта,,,,,,
GBP/RUB,978113,-,977476,"0,07 %",00637,19.09.2020
EUR/RUB,896770,-,889850,"0,78 %",06920,19.09.2020
USD/RUB,756995,-,756502,"0,07 %",00493,19.09.2020
JPY/RUB,07240,-,07235,"0,06 %",00004,19.09.2020
CNY/RUB,111834,-,111760,"0,07 %",00074,19.09.2020
CHF/RUB,830603,-,829953,"0,08 %",00650,19.09.2020
EUR/USD,11843,-,11840,"0,03 %",00004,19.09.2020
EUR/GBP,09165,-,09130,"0,39 %",00035,19.09.2020
EUR/JPY,1238250,-,1238000,"0,02 %",00250,19.09.2020
